In [23]:
from time import time
import lightgbm as lgb
import numpy as np
import sys
# sys.path.append('..\Python Scripts\pipeline')
import warnings
warnings.filterwarnings('ignore') 
import seaborn as sns
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.model_selection import KFold, TimeSeriesSplit
from scipy.stats import uniform
import datetime
from sklearn.ensemble import RandomForestClassifier
import lightgbm
from scipy import stats
# from hyperopt import hp, tpe
# from hyperopt.fmin import fmin
from sklearn.model_selection import cross_val_score, StratifiedKFold
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, \
cross_val_score, GridSearchCV, validation_curve, KFold, GroupKFold
from sklearn.metrics import roc_auc_score
import seaborn as sns
%matplotlib inline
import gc

In [17]:
data = pd.read_pickle(r'f:\my\Prog\kaggle\Baydin\Snapshots\1\01-JoinReduceMem\data.pkl')

# New

In [31]:
for col in ['DT_M', 'DT_W', 'DT_D']:
    temp_df = data[[col]]
    fq_encode = temp_df[col].value_counts().to_dict()

    data[col + '_total'] = data[col].map(fq_encode)

In [ ]:
# ВЫЗОВ ФУНКЦИИ АГРЕГАТ НАД АГРЕГАТОМ

i_cols = ['card3', 'card5', 'bank_type']

data = bank_type_aggregate(data, i_cols, big_period, small_period, type_aggregate)


In [ ]:

# ХОЧЕТСЯ ЕЩЕ ГРУП БАЙ С ВРЕМЕНЕМ ДЛЯ ТРАМТ

# Final addings: new_card_id, time_freq, aggs, additional feats

In [ ]:
c8 = pd.read_pickle(r'f:\my\Prog\kaggle\Baydin\Snapshots\1\NewId\C8_agg.pkl')
c5 = pd.read_pickle(r'f:\my\Prog\kaggle\Baydin\Snapshots\1\NewId\C5_agg.pkl')
tramt = pd.read_pickle(r'f:\my\Prog\kaggle\Baydin\Snapshots\1\NewId\TransactionAmt_agg.pkl')

train = data[data['isFraud'] != -1]
test = data[data['isFraud'] == -1].drop('isFraud', axis=1)
y = data[data['isFraud'] != -1]['isFraud']

train = train.join(c8).join(c5).join(tramt)
test = test.join(c8).join(c5).join(tramt)

In [ ]:
del data
gc.collect()


cols = "TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,M1,M2,M3,M4,M5,M6,M7,M8,M9".split(",")
train_test = train[cols].append(test[cols])

for col in "ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14".split(","):
    col_count = train_test.groupby(col)['TransactionDT'].count()
    train[col+'_count'] = train[col].map(col_count)
    test[col+'_count'] = test[col].map(col_count)


for col in "card1,card2,card5,addr1,addr2".split(","):
    col_count = train_test.groupby(col)['TransactionAmt'].mean()
    train[col+'_amtcount'] = train[col].map(col_count)
    test[col+'_amtcount'] = test[col].map(col_count)
    col_count1 = train_test[train_test['C5'] == 0].groupby(col)['C5'].count()
    col_count2 = train_test[train_test['C5'] != 0].groupby(col)['C5'].count()
    train[col+'_C5count'] = train[col].map(col_count2) / (train[col].map(col_count1) + 0.01)
    test[col+'_C5count'] = test[col].map(col_count2) / (test[col].map(col_count1) + 0.01)
    
del train_test
gc.collect()

In [ ]:
# train.drop([x for x in train.columns if 'by_card_id_ws' in x], axis=1, inplace=True)
# test.drop([x for x in test.columns if 'by_card_id_ws' in x], axis=1, inplace=True)

# train.drop(['Date', 'TransactionDT'], axis=1, inplace=True)
# test.drop(['Date', 'TransactionDT'], axis=1, inplace=True)

# train.drop(['DT_W', 'DT_D'] , axis=1, inplace=True)
# test.drop(['DT_W', 'DT_D'] , axis=1, inplace=True)

In [ ]:
# for x in train.columns:
#     if train[x].dtype == 'O':
#         print(x)

# train.drop('version_id_31', axis=1, inplace=True)
# test.drop('version_id_31', axis=1, inplace=True)

# train.drop('BrowserVersion', axis=1, inplace=True)
# test.drop('BrowserVersion', axis=1, inplace=True)

# train['screen_height'] = train['screen_height'].replace('None', np.nan).astype(float)
# test['screen_height'] = test['screen_height'].replace('None', np.nan).astype(float)

# train['screen_width'] = train['screen_width'].replace('None', np.nan).astype(float)
# test['screen_width'] = test['screen_width'].replace('None', np.nan).astype(float)

In [ ]:
# trainaggs = pd.DataFrame(index=train.index)
# testaggs = pd.DataFrame(index=test.index)

# for item in [
    
#  'C4_aggby_new_card_id.pkl',
#  'C9_aggby_new_card_id.pkl',
#  ]:
#     temp = pd.read_pickle(r'f:\my\Prog\kaggle\Baydin\Snapshots\1\NewId' + '//' + item)
#     trainaggs = trainaggs.join(temp)
#     testaggs = testaggs.join(temp)

# trainaggs = trainaggs[['C4_by_new_card_id_ws3d_to_mean', 'C4_by_new_card_id_ws7d_to_mean',
#        'C4_by_new_card_id_ws30d_to_mean', 'C4_by_new_card_id_ws100_to_mean', 'C9_by_new_card_id_ws7d_to_mean',
#        'C9_by_new_card_id_ws5_to_mean', 'C9_by_new_card_id_ws10_to_mean']]
# testaggs = testaggs[['C4_by_new_card_id_ws3d_to_mean', 'C4_by_new_card_id_ws7d_to_mean',
#        'C4_by_new_card_id_ws30d_to_mean', 'C4_by_new_card_id_ws100_to_mean', 'C9_by_new_card_id_ws7d_to_mean',
#        'C9_by_new_card_id_ws5_to_mean', 'C9_by_new_card_id_ws10_to_mean']]

# train = train.join(trainaggs)
# test = test.join(testaggs)

# del temp, trainaggs, testaggs, train_idx, test_idx
# gc.collect()

In [ ]:
train.drop([x for x in train.columns if 'V' in x], axis=1, inplace=True)
test.drop([x for x in test.columns if 'V' in x], axis=1, inplace=True)

vs = pd.read_pickle(r'f:\my\Prog\kaggle\Baydin\Snapshots\1\VFeatures\v_features_reduced.pkl')

train = train.join(vs)
test = test.join(vs)

In [ ]:
newid = pd.read_pickle(r'f:\my\Prog\kaggle\Baydin\Snapshots\1\NewId\new_id.pkl')

train = train.join(newid)
test = test.join(newid)

train['start_date'] = pd.to_timedelta(pd.to_datetime(train['start_date'])).map(lambda x: x.days)
test['start_date'] = pd.to_timedelta(pd.to_datetime(test['start_date'])).map(lambda x: x.days)

In [ ]:
timefreq = pd.read_pickle(r'f:\my\Prog\kaggle\Baydin\Snapshots\1\TimeAggs24.09\transaction_frequencies.pkl')

train = train.join(timefreq)
test = test.join(timefreq)

train.drop('isFraud', axis=1, inplace=True)

del timefreq, vs
gc.collect()